# Generating human 'attention' from source / summary input.

load the data

In [5]:
import json
f = open('/Users/haldenl/nlpcapstone/data/attn_vis_data.json', 'r')
data = json.load(f)

print(data[0].keys())

dict_keys(['p_gens', 'article_lst', 'abstract_str', 'decoded_lst', 'attn_dists'])


process

In [6]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [7]:
import numpy as np
np.set_printoptions(threshold=np.inf)


softmax function, from @nolanbconaway

In [8]:
def softmax(X, theta = 1.0, axis = None):
    """
    Compute the softmax of each element along an axis of X.

    Parameters
    ----------
    X: ND-Array. Probably should be floats. 
    theta (optional): float parameter, used as a multiplier
        prior to exponentiation. Default = 1.0
    axis (optional): axis to compute values along. Default is the 
        first non-singleton axis.

    Returns an array the same size as X. The result will sum to 1
    along the specified axis.
    """

    # make X at least 2d
    y = np.atleast_2d(X)

    # find axis
    if axis is None:
        axis = next(j[0] for j in enumerate(y.shape) if j[1] > 1)

    # multiply y against the theta parameter, 
    y = y * float(theta)

    # subtract the max for numerical stability
    y = y - np.expand_dims(np.max(y, axis = axis), axis)

    # exponentiate y
    y = np.exp(y)

    # take the sum along the specified axis
    ax_sum = np.expand_dims(np.sum(y, axis = axis), axis)

    # finally: divide elementwise
    p = y / ax_sum

    # flatten if X was 1D
    if len(X.shape) == 1: p = p.flatten()

    return p

### Calculates the similarity betweens sentences in article and summary (softmaxed)

In [9]:
def getSentenceSimilarities(article_sentences, human_sentences):
    sentence_similarities = np.zeros([len(human_sentences), len(article_sentences)])
    
    
    for human_index, human_sent in enumerate(human_sentences):
        for article_index, article_sent in enumerate(article_sentences):
            a_sent = article_sent
            h_sent = human_sent
            
            similarity = h_sent.similarity(a_sent)
            sentence_similarities[human_index][article_index] = similarity
        
    sentence_similarities = softmax(sentence_similarities, theta=20, axis=1)
    return sentence_similarities
    

### Calculates 'attention' weights between article and summary

In [10]:
def getWeights(article, human):
    weights = np.zeros([len(human), len(article)])
    
    article_sentences = list(article.sents)
    human_sentences = list(human.sents)
        
    sentence_similarities = getSentenceSimilarities(article_sentences, human_sentences)
    
    count = 0
    human_rows = {}
    for human_index, human_sent in enumerate(human_sentences):
        human_rows[human_index] = {}
        for human_tok_index, human_token in enumerate(human_sent):
            human_rows[human_index][human_tok_index] = count
            count += 1
            
    count = 0
    article_columns = {}
    for article_index, article_sent in enumerate(article_sentences):
        article_columns[article_index] = {}
        for article_tok_index, article_token in enumerate(article_sent):
            article_columns[article_index][article_tok_index] = count
            count += 1
    
    for human_index, human_sent in enumerate(human_sentences):
        human_sent = nlp(human_sent.text)
        
        for article_index, article_sent in enumerate(article_sentences):
            article_sent = nlp(article_sent.text)
        
            sentence_sim = sentence_similarities[human_index][article_index]
            
            for human_tok_index, human_token in enumerate(human_sent):
                count += 1
                for article_tok_index, article_token in enumerate(article_sent):
                    similarity = 0
                    if (len(human_token.text) == 1 or len(article_token.text) == 1):
                        similarity = 0
                    else:
                        similarity = human_token.similarity(article_token)
                        
                        
                    weight = sentence_sim * similarity
    
                    row = human_rows[human_index][human_tok_index]
                    column = article_columns[article_index][article_tok_index]
                    weights[row][column] = sentence_sim                  
                  
    weights = softmax(weights, theta=100, axis=1)
    
    return weights

In [11]:
article_str="It's complicated, but there I was in a Sandals resort in Grenada, not with my wife Gillian but with my best friend from university, Charlotte Metcalf. By coincidence we had pitched up together to review the 14th in the Sandals chain. Neither of us had been before, but Couples Only? Weddings on the beach? Charlotte and I, both hard-bitten travellers, were set to sneer. Sandals La Source has 225 suites spread over three ‘villages’ in a small sandy bay. On arrival Ritchie, the manager, tells me his philosophy is ‘to give guests more than they expect’, and at first glance it’s hard to tell who’s happier – the 350 guests or the 500 staff. Certainly everyone appears to be grinning constantly, as if part of a cult - the Sandalistas. Among Sandals LaSource Grenada's 225 suites is the Skypool, with this stunning private infinity pool The three 'village' luxury all-inclusive resort hosts 350 guests with 500 staff Our personal butlers, trained by the Guild of Professional English Butlers, take us to our (separate) rooms. Charlotte’s giant suite features a black marble bath and a terrace with plunge pool and Jacuzzi. Meanwhile, I follow my butler, Marcus, up to my Oceanview Skypool Penthouse. Marcus hands me a mobile telephone and says: ‘Anything you need, I’m a phone call away.’ From my private infinity pool, I look out over the fort, and reflect on what I know about Grenada. It is mountainous, measuring just 12 miles by 21, with a population of 110,000, a high level of literacy and land ownership, and one of the lowest crime figures in the Caribbean. The island’s recent peaceful history has emerged from two tempests. One was a natural phenomenon, Hurricane Ivan, that struck in September 2004 and destroyed 85 per cent of houses and vegetation, including Grenada’s precious nutmeg trees. Sandals La Source is situated on a sandy bay on mountainous Grenada, an island just 12 by 21 miles Among the resort's shared facilities is three freshwater swimming pools including this in the 'Pink Gin' village The second was political: the 1983 Marxist coup in which Prime Minister Maurice Bishop was killed – a move that prompted Ronald Reagan to land 6,000 US troops on what was part of the Commonwealth. My father was a Foreign Office diplomat at the time, and he recalls meeting the Queen shortly after the invasion. She was furious. Charlotte and I find the scale of the resort overwhelming. There are nine restaurants, several bars, and even an English pub. After some fishing, during which I catch and release a 200lb blue marlin, I land an audience with the island’s popular Prime Minister. Dr Keith Mitchell greets us barefoot at his official residence, where we chat about Grenada’s future as an eco-tourist centre and its current peace and stability, which he ascribes to his people’s inherently cheerful nature. The Sandals resort occupies the heart of Grenada's exclusive Pink Gin Beach Writer Nicholas Shakespeare enjoys a pedalo ride with his friend Charlotte Metcalf The South Seas Butler Suite's living room opens up to a trellised veranda with a private plunge pool Back at La Source, Charlotte and I attend a beach ‘Re-Tie The Knot’ ceremony. Two couples, from Halifax and Leicester, both married for more than 20 years, arrive for a moving service conducted with palpable delight by Baptist pastor Stanford Simon, looking splendid in lime-green linen. Sandals (0800 597 0002) offers seven nights at Sandals LaSource Grenada Resort & Spa from £1,629pp. This includes Luxury Included (all-inclusive) accommodation in a South Seas Hideaway Junior Suite with Balcony Tranquility Soaking Tub, return flights from London and resort transfers. Valid for travel between September 1 and October 20, 2015. There are tears of joy as well as laughter as Karen and Jenny, both in full-length cream and carrying white orchids, walk up the jetty to a small pavilion hanging over the sea to repledge their love to husbands Phil and Kieron respectively. The staff are in white waistcoats and ties for the occasion. The couples have prepared statements to read to each other, giving thanks for the love and lives they share. It is the simplest, most genuine and therefore the most profoundly moving service I’ve been to… well, since my own wedding. Charlotte and I weep openly. Sandals is victorious – cynicism is slain."
human_str="Sandals La Source in Grenada has 225 suites spread over three ‘villages’ Personal butlers are trained by the Guild of Professional English Butlers The resort in a small sandy bay has 500 staff tending to its 350 guests"

In [12]:
article = nlp(article_str)  
human = nlp(human_str)

weights = getWeights(article, human)

attentionRecords = []
inputRecords = []
outputRecords = []

for human_index, human_token in enumerate(human):
    for article_index, article_token in enumerate(article):
        
        attn_weight = weights[human_index][article_index]
                
        attentionRecords.append({
            'inputIndex': article_index,
            'outputIndex': human_index,
            'weight': attn_weight
        })
        
for human_index, human_token in enumerate(human):
    outputRecords.append({
        'index': human_index,
        'token': human_token.text
    })
    
for article_index, article_token in enumerate(article):
    inputRecords.append({
        'index': article_index,
        'token': article_token.text
    })
            
output = {
    'attentionRecords': attentionRecords,
    'inputTokens': inputRecords,
    'outputTokens': outputRecords
}

with open('/Users/haldenl/nlpcapstone/data/hierarchical_similarity_data_{0}.json'.format(0), 'w') as out:
    json.dump(output, out, indent=2)
